In [0]:
%sql
DROP SCHEMA IF EXISTS mlops.data CASCADE;
DROP SCHEMA IF EXISTS mlops.model CASCADE;

In [0]:
%sql
create schema mlops.data;
create schema mlops.model;

In [0]:
%sql
SELECT
  MIN(order_datetime) AS min_order_datetime,
  MAX(order_datetime) AS max_order_datetime
FROM test_catalog.default.sales_orders_customer_join_min

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.batch_recommendations_topn_30d

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.als_interactions_30d

In [0]:
%sql
SELECT b.* 
FROM test_catalog.default.sales_orders_bronze b
LEFT ANTI JOIN databricks_simulated_retail_customer_data.v01.sales_orders r
ON b.order_number = r.order_number;


In [0]:
%sql
SELECT r.* 
FROM databricks_simulated_retail_customer_data.v01.sales_orders r
LEFT ANTI JOIN test_catalog.default.sales_orders_bronze b
ON r.order_number = b.order_number;

In [0]:
%sql
SELECT b.* 
FROM test_catalog.default.customers_bronze b
LEFT ANTI JOIN databricks_simulated_retail_customer_data.v01.customers r
ON b.customer_id = r.customer_id;


In [0]:
%sql
SELECT b.* 
FROM test_catalog.default.customers_bronze b
LEFT ANTI JOIN databricks_simulated_retail_customer_data.v01.customers r
ON b.customer_id = r.customer_id;

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.sales_orders_customer_join_min

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.customers_bronze

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.customers_silver

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.sales_orders_bronze

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.sales_orders_silver

In [0]:
%sql
SELECT
  *
FROM test_catalog.default.sales_orders_bronze

In [0]:
%sql
SELECT
  *
FROM 

In [0]:
from pyspark.sql import functions as F, types as T

ITEMS_SCHEMA = T.ArrayType(
    T.StructType([
        T.StructField("curr", T.StringType()),
        T.StructField("id",   T.StringType()),
        T.StructField("name", T.StringType()),
        T.StructField("price",T.StringType()),
        T.StructField("promotion_info", T.StringType()),
        T.StructField("qty",  T.StringType()),
        T.StructField("unit", T.StringType()),
    ])
)

base = spark.read.table(
    "test_catalog.default.sales_orders_customer_join_min"
).alias("base")

j = (
    base.withColumn(
        "ts",
        F.to_timestamp(
            F.from_unixtime(
                F.col("order_datetime").cast("bigint")
            )
        )
    )
    .withColumn(
        "items",
        F.from_json("ordered_products", ITEMS_SCHEMA)
    )
    .withColumn(
        "p",
        F.explode("items")
    )
)

mx = j.agg(F.max("ts").alias("max_ts"))
j = j.crossJoin(mx)
w30 = F.col("ts") >= F.date_sub(F.col("max_ts"), 30)

result = (
    j.where(w30)
    .select(
        "customer_id",
        F.col("p.id").alias("product_id"),
        F.col("p.qty").cast("int").alias("qty")
    )
    .where(
        F.col("qty").isNotNull() & (F.col("qty") > 0)
    )
    .groupBy("customer_id", "product_id")
    .agg(
        F.sum("qty").alias("total_qty")
    )
    .withColumn(
        "interaction_weight",
        F.log1p(F.col("total_qty"))
    )
)

display(result)